<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part2_basic_rag_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will develop url RAG using langchain. Also, we will use google gemini api key for llm and cohere api key for embeddings.

**Requirements.txt**

In [2]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo cohere >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo llama-index >> requirements.txt
!echo pypdf >> requirements.txt
!echo chromadb >> requirements.tx
!echo beautifulsoup4 >> requirements.tx
!echo matplotlib >> requirements.tx
!echo rank_bm25 >> requirements.tx
!echo replicate >> requirements.txt

**Bash/command**

In [3]:
pip install -r requirements.txt

  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings
import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_google = os.getenv("google_apikey")
#my_key_cohere = os.getenv("cohere_apikey")

#for google colab
my_key_google="---"
my_key_cohere="----"

llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-pro")
embeddings = CohereEmbeddings(cohere_api_key=my_key_cohere,model="embed-multilingual-v3.0")

def ask_gemini(prompt):

    AI_Response = llm_gemini.invoke(prompt)

    return AI_Response.content

def rag_with_url(target_url, prompt):
  loader=WebBaseLoader(target_url)
  raw_documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=0,
      length_function=len
  )

  splitted_documents = text_splitter.split_documents(raw_documents)

  vectorstore = FAISS.from_documents(splitted_documents,embeddings)
  retriever = vectorstore.as_retriever()

  relevant_documents = retriever.get_relevant_documents(prompt)

  context_data= ""

  for document in relevant_documents:
    context_data = context_data + " " + document.page_content

  final_prompt = f"""Şöyle bir sorum var: {prompt}
  Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
  Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
  """

  AI_Response = ask_gemini(prompt=final_prompt)

  return AI_Response, relevant_documents

test_url = "https://kpmg.com/tr/tr/home/gorusler/2023/12/uretken-yapay-zeka-uygulamalarinin-kurumsallasma-yaklasimi.html"

test_question = "Üretken yapay zeka uygulamalarının hayata geçirirken yaşanan temel sorunlar neler?"

AI_Response, relevant_documents = rag_with_url(target_url=test_url, prompt=test_question)


print(f'SORU: {test_question}')
print("-"*100)
print(f'YZ YANITI: {AI_Response}')
print("-*100")
for doc in relevant_documents:
  print(doc.page_content)
  print("-"*100)
  print(doc.metadata)
  print("*"*100)




SORU: Üretken yapay zeka uygulamalarının hayata geçirirken yaşanan temel sorunlar neler?
----------------------------------------------------------------------------------------------------
YZ YANITI: Verilen bilgiler ışığında, üretken yapay zeka uygulamalarının hayata geçirirken yaşanan temel sorunlar şunlardır:

* **Yetenek Kazanımı:** Yeterli sayıda yetkin yapay zeka mühendisi bulma zorluğu.
* **Yapay Zekaya Güven:** Üretken yapay zekanın güvenilir olmayan sonuçlar üretme potansiyeli.
-*100
Yetenek Kazanımı: Veri bilimcilerin bile kurumlarda tutundurulmasının zor olduğu bir dünyada üretken yapay zeka alanında yetkinlik sahibi olan yapay zeka mühendislerinin azlığı, sürdürülebilir bir ekip kurma ve bu alanda yeni geliştirmelerin yapılmasında büyük zorluklar yaşattı. 
Yapay Zekaya Güven: Üretken yapay zekanın ürettiği sonuçların operasyonel bir sistemin karar verme alma noktalarına entegre edildiği noktada, sonucun tutarlılık ve açıklanabilirliği ile ilgili yarattığı güven kaygısı, kr